<h3>Setup</h3>

In [ ]:
# Libraries
import torch
import sys
import platform


# MacOs
if platform.system() == "Darwin":
    main_dir = './'
    data_dir = '../data/'
    weights_dir = '../weights/'
    
    device = 'cpu'
    #device = 'mps' if torch.backends.mps.is_available()  else 'cpu'
    
    print(f"System: MacOs\nDevice: {device}\nMain_dir: {main_dir}")

# Linux
elif platform.system() == "Linux" and 'google.colab' not in sys.modules:
    main_dir = './'
    data_dir = '../data/'
    detector_weights_dir = '../weights/'
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    print(f"System: Linux\nDevice: {device}\nMain_dir: {main_dir}")

# Colab
elif platform.system() == "Linux" and 'google.colab' in sys.modules:
    !pip install pytesseract
    from google.colab import drive
    
    drive.mount('/content/drive')
    
    main_dir = '/content/drive/MyDrive/Università/Artificial Intelligence and Robotics/1° Anno/2° Semestre/Vision and Perception/Project/'
    data_dir = 'data/'
    detector_weights_dir = 'weights/'
    
    sys.path.insert(0, main_dir)
    sys.path.insert(1, main_dir+'src/')
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    print(f"System: Colab\nDevice: {device}\nMain_dir: {main_dir}")

# Unknown
else:
    print("System unknown")


<h3>Prepare datasets and loaders</h3>

In [ ]:
from modules import dataset

PREPARE_CSV = False
PLOT_STATS = False

In [ ]:
if PREPARE_CSV: 
    df_closeup = dataset.xml2csv_close_up(path_folder=main_dir+data_dir+'close_up/')
    df_traffic_camera = dataset.xml2csv_traffic_camera(path_folder=main_dir+data_dir+'traffic_camera/')

In [ ]:
train_ds_cu, valid_ds_cu = dataset.get_datasets(data_dir=main_dir+data_dir+'close_up/', csv_file='dataset.csv')
train_ds_tc, valid_ds_tc = dataset.get_datasets(data_dir=main_dir+data_dir+'traffic_camera/', csv_file='dataset.csv')

In [ ]:
if PLOT_STATS: dataset.dataset_stats(dataset=train_ds_cu)
if PLOT_STATS: dataset.dataset_stats(dataset=train_ds_tc)

In [ ]:
dataset.plot_image_from_dataset(dataset=train_ds_cu, idx=0)

In [ ]:
dataset.plot_image_from_dataset(dataset=train_ds_tc, idx=0)

In [ ]:
train_dl_cu, valid_dl_cu = dataset.get_dataloaders(train_ds=train_ds_cu, valid_ds=valid_ds_cu, batch_size=8)
train_dl_tc, valid_dl_tc = dataset.get_dataloaders(train_ds=train_ds_tc, valid_ds=valid_ds_tc, batch_size=8)

<h3>Training</h3>

In [ ]:
TRAIN_MODEL=False
LOAD_MODEL=False
DATA_LOADER='traffic_camera'

if DATA_LOADER == 'close_up':
    train_dl = train_dl_cu
    valid_dl = valid_dl_cu
elif DATA_LOADER == 'traffic_camera':
    train_dl = train_dl_tc
    valid_dl = valid_dl_tc

In [ ]:
if TRAIN_MODEL:
    from modules import model, training
    
    detector = model.get_plateDetectorModel(num_classes=3, feature_extraction=True)
    if LOAD_MODEL:
        detector.load_state_dict(torch.load(main_dir+weights_dir+'detector_weights/detector.pt'))

    optimizer = torch.optim.SGD([p for p in detector.parameters() if p.requires_grad],
                                lr = 0.005,
                                momentum = 0.9,
                                weight_decay = 0.0005)
    
    losses = training.training_and_validation(model=detector.to(device),
                                                optimizer=optimizer,
                                                num_epochs=50,
                                                train_loader=train_dl,
                                                valid_loader=valid_dl,
                                                device=device,
                                                weights_dir=main_dir+weights_dir+'detector_weights/',
                                                verbose=2)
    
    training.plot_losses(train_epoch_losses=losses[0], valid_epoch_losses=losses[1])

<h3>Inference</h3>

In [ ]:
from modules import inference, real_time_object_detector

COMPUTE_REAL_TIME_OBJECT_DETECTOR = True
FRAMES_TO_VIDEO = False

In [ ]:
if COMPUTE_REAL_TIME_OBJECT_DETECTOR: 
    import cv2

    detector = inference.load_model_for_inference(weights_file=main_dir+weights_dir+'detector_weights/detector.pt', device=device)
    cv2_vehicles_cfg = {'fontFace':cv2.FONT_HERSHEY_SIMPLEX, 'fontScale':0.65, 'color':(0,0,255), 'thickness':2, 'lineType':cv2.LINE_AA}
    cv2_plates_cfg = {'fontFace':cv2.FONT_HERSHEY_SIMPLEX, 'fontScale':0.6, 'color':(255,0,0), 'thickness':2, 'lineType':cv2.LINE_AA}
    velocity_cfg = {'line1': 1025, 'line2': 1205, 'meters_line1line2':7.5, 'fps':25}
    
    real_time_object_detector.real_time_object_detector(model=detector, 
                                                        video_path='../video_test/video_test.mp4', 
                                                        sr_weights_path=weights_dir+'edsr_weights/EDSR_x3.pb',
                                                        spl_weights_path=weights_dir+'alpr_weights/ALPR_weights.pth.zip',
                                                        velocity_cfg=velocity_cfg,
                                                        cv2_vehicles_cfg=cv2_vehicles_cfg, 
                                                        cv2_plates_cfg=cv2_plates_cfg,
                                                        new_frame_folder=main_dir+'../output_video/',
                                                        frames_to_skip=130)